# Multivariate statistics (decoding / MVPA) on MEG/EEG

Authors : Alexandre Gramfort, Richard Höchenberger

See more info on decoding on this page: https://mne.tools/stable/auto_tutorials/machine-learning/plot_sensors_decoding.html

First, load the mne package:

In [ ]:
import mne

We set the log-level to 'WARNING' so the output is less verbose

In [ ]:
mne.set_log_level('WARNING')

## Access the raw data

Now we import the `sample` dataset. It will be downloaded automatically (approx. 2 GB).

In [ ]:
import os
from mne.datasets import sample
data_path = sample.data_path()

raw_fname = os.path.join(data_path, 'MEG/sample/sample_audvis_filt-0-40_raw.fif')
raw = mne.io.read_raw_fif(raw_fname, preload=True)
raw

High-pass filter the data.

In [ ]:
raw.filter(l_freq=1, h_freq=None, verbose=True)

In [ ]:
print(raw.info)

## Define epochs

We're only interested in `auditory left` and `auditory right`.

First extract events:

In [ ]:
events = mne.find_events(raw, stim_channel='STI 014', verbose=True)

Look at the design in a graphical way. We're only interested in `auditory left` and `auditory right`.

In [ ]:
event_id = {'aud_l': 1, 'aud_r': 2}  # event trigger and conditions
fig = mne.viz.plot_events(events, sfreq=raw.info['sfreq'],
                          first_samp=raw.first_samp, event_id=event_id)

Define epochs parameters:

In [ ]:
tmin = -0.1  # start of each epoch (in seconds)
tmax = 0.4   # end of each epoch
baseline = None  # no baseline correction, data were high-pass filtered

reject = dict(eeg=80e-6, eog=40e-6)
picks = mne.pick_types(raw.info, eeg=True, meg=True,
                       eog=True, stim=False, exclude='bads')

epochs = mne.Epochs(raw, events, event_id, tmin, tmax, proj=True,
                    picks=picks, baseline=baseline,
                    reject=reject, preload=True)  # with preload

print(epochs)

Look at the ERF and contrast between left and right stimulation.

In [ ]:
evoked_left = epochs['aud_l'].average()
evoked_right = epochs['aud_r'].average()
evoked_contrast = mne.combine_evoked([evoked_left, evoked_right],
                                     weights=[1, -1])

In [ ]:
fig = evoked_left.plot()
fig = evoked_right.plot()
fig = evoked_contrast.plot()

### Plot some topographies

In [ ]:
vmin, vmax = -4, 4  # Colorbar range
fig = evoked_left.plot_topomap(ch_type='eeg', contours=0, vmin=vmin, vmax=vmax)
fig = evoked_right.plot_topomap(ch_type='eeg', contours=0, vmin=vmin, vmax=vmax)
fig = evoked_contrast.plot_topomap(ch_type='eeg', contours=0, vmin=None, vmax=None)

## Now let's see if we can classify single trials.

To keep chance level at 50% accuracy, we first equalize the number of epochs in each condition.

In [ ]:
epochs.equalize_event_counts(event_id)
print(epochs)

A classifier takes as input an `X` and returns `y` (0 or 1). Here `X` will be the data at one time point on all gradiometers (hence the term multivariate). We want to train our model to discriminate between the  `auditory left` and the `auditory right` trials.

We work with all sensors jointly and try to find a discriminative pattern between the two conditions to predict the class.

For classification we will use the scikit-learn package (http://scikit-learn.org/) and MNE functions.

Let's first create the response vector, `y`.

In [ ]:
import numpy as np

y = np.empty(len(epochs.events), dtype=int)
idx_auditory_left = epochs.events[:, 2] == event_id['aud_l']
idx_auditory_right = epochs.events[:, 2] == event_id['aud_r']
y[idx_auditory_left] = 0
y[idx_auditory_right] = 1

y.size

Now, the input matrix, `X`.

In [ ]:
X = epochs.copy().pick_types(meg='grad').get_data()
X.shape

In [ ]:
XX = X.reshape(108, -1)
XX.shape

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

logreg = LogisticRegression(solver='liblinear')
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
clf = make_pipeline(StandardScaler(), logreg)

scores = cross_val_score(clf, XX, y, cv=cv, scoring='roc_auc')

roc_auc_mean = np.mean(scores)
roc_auc_std = np.std(scores)

print(f'CV scores: {scores}')
print(f'Mean ROC AUC = {roc_auc_mean:.3f} (std: {roc_auc_std:.3f})')

## We can do this more simply using the `mne.decoding` module! Let's do some spatio-temporal decoding.

In [ ]:
from sklearn.pipeline import make_pipeline
from mne.decoding import Scaler, Vectorizer, cross_val_multiscore

epochs_decoding = epochs.copy().pick_types(meg='grad')

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
clf = make_pipeline(Scaler(epochs_decoding.info),
                    Vectorizer(),
                    logreg)

X = epochs_decoding.get_data()
y = epochs_decoding.events[:, 2]

scores = cross_val_multiscore(clf, X, y, cv=cv, scoring='roc_auc')

roc_auc_mean = np.mean(scores)
roc_auc_std = np.std(scores)

print(f'CV scores: {scores}')
print(f'Mean ROC AUC = {roc_auc_mean:.3f} (std: {roc_auc_std:.3f})')

<div class="alert alert-success">
    <b>EXERCISES:</b>
     <ul>
      <li>Why do you get different results from above? what did we change in the model?</li>
      <li>How does the choice of cross-validation affect the results? Hint: Change the random_state</li>
      <li>Try a different cross-validtion object like scikit-learn StratifiedShuffleSplit</li>
      <li>Which sensor types give the best classification scores? EEG, MEG gradiometers, MEG magnetometers?</li>
    </ul>
</div>

In [ ]:
from sklearn.pipeline import make_pipeline
from mne.decoding import Scaler, Vectorizer, cross_val_multiscore

for ch_type in ['grad', 'mag', 'eeg']:
    epochs_decoding = epochs.copy().pick(ch_type)

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    clf = make_pipeline(Scaler(epochs_decoding.info),
                        Vectorizer(),
                        logreg)

    X = epochs_decoding.get_data()
    y = epochs_decoding.events[:, 2]

    scores = cross_val_multiscore(clf, X, y, cv=cv, scoring='roc_auc')

    roc_auc_mean = np.mean(scores)
    roc_auc_std = np.std(scores)

    print(f'{ch_type} -- Mean ROC AUC = {roc_auc_mean:.3f} (std: {roc_auc_std:.3f})')

## Decoding over time: Comparisons at every single time point.

In the previous examples, we have trained a classifier to discriminate between experimentel conditions by using the spatio-temporal patterns of **entire trials**. Consequently, the classifier was (hopefully!) able to predict which activation pattern belonged to which condition. 

However, an interesting neuroscientific is: **Exactly *when* do the brain signals for two conditions differ?**

We can try to answer this question by fitting a classifier **at every single time point.** If the classifier can successfully discriminate between the two conditions, we can conclude that the spatial activation patterns measured by the MEG or EEG sensors differed **at this time point**.

In [ ]:
from sklearn.preprocessing import StandardScaler
from mne.decoding import SlidingEstimator

X = epochs_decoding.get_data()
y = epochs_decoding.events[:, 2]


clf = make_pipeline(StandardScaler(),
                    logreg)

time_decod = SlidingEstimator(clf, scoring='roc_auc', n_jobs=1, verbose=True)
scores = cross_val_multiscore(time_decod, X, y, cv=5, n_jobs=1)

# Mean scores across cross-validation splits, for each time point.
mean_scores = np.mean(scores, axis=0)

# Mean score across all time points.
mean_across_all_times = np.mean(scores)
print(f'Mean CV score across all time points: {mean_across_all_times:.3f}')

# Plot
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.plot(epochs.times, mean_scores, label='score')
ax.axhline(0.5, color='k', linestyle='--', label='chance')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Mean ROC AUC')
ax.legend()
ax.axvline(0, color='k', linestyle='-')
ax.set_title('Sensor space decoding')

For more details see: https://mne.tools/stable/auto_tutorials/machine-learning/50_decoding.html

and this book chapter:

Jean-Rémi King, Laura Gwilliams, Chris Holdgraf, Jona Sassenhagen, Alexandre Barachant, Denis Engemann, Eric Larson, Alexandre Gramfort. Encoding and Decoding Neuronal Dynamics: Methodological Framework to Uncover the Algorithms of Cognition. 2018. https://hal.archives-ouvertes.fr/hal-01848442/

<div class="alert alert-success">
    <b>EXERCISES:</b>
     <ul>
      <li>Plot the decoding score over time for the different channel types.</li>
      <li>Do a decoding over time on the SPM `face` dataset to see if you can classify `face` vs. `scrambled face`.</li>
         <li>Do a generalization over time analysis as explained in the <a href="https://mne.tools/stable/auto_tutorials/machine-learning/plot_sensors_decoding.html#temporal-generalization">documentation on decoding</a>.
</li>
    </ul>
</div>

Hints:

- Access the `face` dataset via:

    ```
    from mne.datasets import spm_face
    data_path = spm_face.data_path()

    raw_fname = os.path.join(data_path, 'MEG/spm/SPM_CTF_MEG_example_faces1_3D.ds')
    raw = mne.io.read_raw_ctf(raw_fname, preload=True)
    ```

- The event IDs are:

    ```
    event_ids = {"faces": 1, "scrambled": 2}
    ```

See this online example for additional hints: https://mne.tools/stable/auto_examples/datasets/spm_faces_dataset_sgskip.html
